In [12]:
from langgraph.graph import StateGraph ,START , END , add_messages
from typing import TypedDict , Annotated
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from pydantic import BaseModel , Field
from langchain_core.messages import BaseMessage , HumanMessage
from langgraph.checkpoint.memory import MemorySaver
import os
load_dotenv()
GROQ_API_KEY = os.getenv('GROQ_API_KEY1')

model = ChatGroq(model_name="llama3-70b-8192", api_key=GROQ_API_KEY, temperature=0.7)


In [3]:
import operator
from langgraph.graph.message import add_messages

In [6]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage] ,add_messages]

In [ ]:
def chat_node(state: ChatState):
    messages = state['messages']

    response = model.invoke(messages).content

    return {'messages': [response]}

In [13]:

checkpointer = MemorySaver()
graph = StateGraph(ChatState)

graph.add_node("chat_node" , chat_node)
graph.add_edge(START , 'chat_node')
graph.add_edge('chat_node' , END)

chatbot = graph.compile(checkpointer=checkpointer)

In [10]:
initial_state = {'messages': [HumanMessage(content='What is the capital of Sikkim')]}
chatbot.invoke(initial_state)['messages'][-1].content

'The capital of Sikkim is Gangtok.'

In [14]:
thread_id = '1'

while True:
    user_message  = input('Type here: ')

    print('User :', user_message)
    if user_message.strip().lower() in ['exit' , 'quit' , 'bye']:
        break
    config = {'configurable': {'thread_id':thread_id}}
    response = chatbot.invoke({'messages': [HumanMessage(content=user_message)]} , config=config)
    print('AI: ' , response['messages'][-1].content)

User : hi
AI:  Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?
User : what is 2 + 2
AI:  The answer to 2 + 2 is 4.
User : add 5 to the result
AI:  If we add 5 to 4, we get:

4 + 5 = 9
User : quit


In [17]:
chatbot.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='16061fa7-c4d0-4746-8d03-d39e6a03b6d9'), AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_time': 0.075968892, 'prompt_time': 0.000145008, 'queue_time': 0.275178314, 'total_time': 0.0761139}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_bf16903a67', 'finish_reason': 'stop', 'logprobs': None}, id='run--fb23a923-7d28-4393-a51c-dd5a2cd4effe-0', usage_metadata={'input_tokens': 11, 'output_tokens': 25, 'total_tokens': 36}), HumanMessage(content='what is 2 + 2', additional_kwargs={}, response_metadata={}, id='890fa635-524a-4712-ae7a-6dea5ef2d4d9'), AIMessage(content='The answer to 2 + 2 is 4.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 